In [1]:
import json, iso8601, fiona

In [2]:
#Load the files
raw_strings = sc.textFile('/data/chime/MovementDerivationGeoGNIP/VulnerableLocations/*/*')

#Filter out the duds
strings = raw_strings.filter(lambda x: x!="")

#JSONs
jsons  = strings.map(json.loads)

tweet_jsons = jsons.filter(lambda x: 'info' not in x.keys())

Yes, for some reason, not all the tweets have a `geo` field. Moreover, the GNIP `geo` field is backwards from convention

In [3]:
tweets_with_geo = tweet_jsons.filter(lambda t: 'geo' in t.keys())

In [4]:
def fix_geo(t):
    t['geo']['coordinates'].reverse()
    return t
geo_tweets = tweets_with_geo.map(fix_geo)
geo_tweets.cache()

PythonRDD[2] at RDD at PythonRDD.scala:53

In [5]:
geo_tweets.take(1)

[{u'actor': {u'displayName': u'Your #2 (;',
   u'favoritesCount': 33,
   u'followersCount': 672,
   u'friendsCount': 990,
   u'id': u'id:twitter.com:331668063',
   u'image': u'https://si0.twimg.com/profile_images/2643005436/5f541b91c7f5ec3adcacc471e8c911ea_normal.jpeg',
   u'languages': [u'en'],
   u'link': u'http://www.twitter.com/Dougieee_Nastyy',
   u'links': [{u'href': None, u'rel': u'me'}],
   u'listedCount': 0,
   u'location': {u'displayName': u"Your Girl's Bedroom ;)",
    u'objectType': u'place'},
   u'objectType': u'person',
   u'postedTime': u'2011-07-08T14:52:11.000Z',
   u'preferredUsername': u'Dougieee_Nastyy',
   u'statusesCount': 7397,
   u'summary': u"- I'm your girls fantasy (; , #TeamGod #Philippians4: 13 #GrindHard #BallisLife . So just follow me and My Everything @B_MooreSeductiv , I'll lead you  . (; ",
   u'twitterTimeZone': None,
   u'utcOffset': None,
   u'verified': False},
  u'body': u'- I do my own thing , you made me this way .',
  u'generator': {u'displayNa

# Export subsets of the data as `.geojsonl` files

In [6]:
#all_geo_tweets = tweets_with_geo.collect()

In [7]:
#write_full_tweets_to_geojsonl('all_geotagged_tweets_full', all_geo_tweets)

In [8]:
#write_bare_tweets_to_geojsonl('all_geotagged_tweets_bare', all_geo_tweets)

Group By User now to avoid headaches later

In [9]:
tweets_gb_user = geo_tweets.groupBy(lambda t: t['actor']['id'])
tweets_gb_user.cache()

PythonRDD[8] at RDD at PythonRDD.scala:53

In [10]:
tweets_gb_user.take(1)

[(u'id:twitter.com:883377476',
  <pyspark.resultiterable.ResultIterable at 0x7f2a6c105e90>)]

In [43]:
def write_out_simplified_geo_contextual(tuple_of_uid_tweets):
    u_tweets = list(tuple_of_uid_tweets[1])
    u_tweets.sort(key=lambda t: iso8601.parse_date(t['postedTime']))
    fileName = u_tweets[0]['actor']['preferredUsername'].lower()
    write_simplified_tweets_to_geojsonl('../working_data/simplified_geo_contextual_all_users/'+fileName,u_tweets)

In [44]:
#Things TODO here: Create a directory of all user files, called the 'geo-contextual'
tweets_gb_user.foreach( write_out_simplified_geo_contextual )

## Geographic Filtering

In [ ]:
from shapely.geometry import mapping, shape
c = fiona.open('../ZoneA_Geometry/ZoneA/OGRGeoJSON.shp','r')
pol = c.next()
geom = shape(pol['geometry']).buffer(0)
geom.is_valid

In [ ]:
zoneABroadcast = sc.broadcast(geom)

In [ ]:
#Testing:
in_bounds = shape({'type': "Point", 'coordinates': [-73.94900, 40.73389]})
print in_bounds
print "TRUE?", zoneABroadcast.value.contains(in_bounds)
print "TRUE?", in_bounds.within(zoneABroadcast.value)

out_of_bounds = shape({"type": "Point","coordinates": [-73.99753114562988,40.73093368341445]})
print out_of_bounds
print "FALSE?", zoneABroadcast.value.contains(out_of_bounds)

In [ ]:
#Filter for those tweets in ZoneA
inZoneA = tweets_with_geo.filter(lambda t: zoneABroadcast.value.contains( shape(t['geo']) ) )

In [ ]:
#Group by user (for inside of Zone A)
inZoneA_gb_user = inZoneA.groupBy(lambda t: t['actor']['id']).collect()

In [ ]:
users_with_at_least_one_tweet_in_zoneA = [u[0] for u in inZoneA_gb_user]
len(users_with_at_least_one_tweet_in_zoneA)

In [ ]:
# json.dump(users_with_at_least_one_tweet_in_zoneA, open('../working_data/users_with_at_least_one_tweet_in_zoneA.json','wb'))

### Write out the ZoneA tweets, but first, ensure it's sorted by time

In [ ]:
print(tweets[0]['postedTime'])
iso8601.parse_date(tweets[0]['postedTime'])

In [ ]:
for (user, tweets) in inZoneA_gb_user:
    tweets = [t for t in tweets]
    userName = tweets[0]['actor']['preferredUsername'].lower()
    write_full_tweets_to_geojsonl('../working_data/tweets_in_zone_a_by_user/'+userName, tweets)

### Get all Tweets from these Users (Beyond just those in ZoneA)

In [45]:
users_with_one_tweet_in_zoneA = json.load(open('../working_data/users_with_at_least_one_tweet_in_zoneA.json'))
len(users_with_one_tweet_in_zoneA)

21951

In [46]:
def write_out_simplified_geo_contextual(tuple_of_uid_tweets):
    if tuple_of_uid_tweets[0] in users_with_one_tweet_in_zoneA:
        u_tweets = list(tuple_of_uid_tweets[1])
        u_tweets.sort(key=lambda t: iso8601.parse_date(t['postedTime']))
        fileName = u_tweets[0]['actor']['preferredUsername'].lower()
        write_full_tweets_to_geojsonl('../working_data/all_tweets_from_zoneA_users/'+fileName,u_tweets)

In [ ]:
#Iterate through the grouped by user tweets and if the uid matches a user with a tweet in zoneA, then write it out!
tweets_gb_user.foreach(write_out_simplified_geo_contextual)

## Temporal Subsets

## Available Export Functions (Requires Local Arrays)
(Also ALWAYS sorts by time; this could be expensive for large collections, but it's important)

In [22]:
def write_full_tweets_to_geojsonl(fileName, tweets):
    with open('../working_data/'+fileName+'.geojsonl','w') as outFile:
#         tweets.sort(key=lambda t: iso8601.parse_date(t['postedTime']))
        for t in tweets:
            geojson = {
                'type':'Feature',
                'geometry':t['geo'],
                'properties':t #Full GNIP Tweet in the properties
            }
            outFile.write(json.dumps(geojson)+"\n")

In [42]:
def write_simplified_tweets_to_geojsonl(fileName, tweets):
    with open('../working_data/'+fileName+'.geojsonl','w') as outFile:
#         tweets.sort(key=lambda t: iso8601.parse_date(t['postedTime']))
        for t in tweets:
            if 'location' in t:
                loc = t['location']
            else:
                loc = None
            if 'location' in t['actor']:
                u_loc = t['actor']['location']
            else:
                u_loc = None
            geojson = {
                'type':'Feature',
                'geometry':t['geo'],
                'properties':{
                    'user':t['actor']['preferredUsername'],
                    'uid' :t['actor']['id'],
                    'u_loc':u_loc,
                    'u_reg':t['actor']['postedTime'],
                    'u_sum':t['actor']['summary'],
                    'tid' :t['id'],
                    'loc' :loc,
                    'time':t['postedTime'],
                    'text':t['body'],
                    'source':t['generator'],
                    'verb':t['verb'],
                    'meta':t['twitter_entities'],
                    'u_utc':t['actor']['utcOffset']
                }
            }
            outFile.write(json.dumps(geojson)+"\n")

In [24]:
def write_bare_tweets_to_geojsonl(fileName, tweets):
    with open('../working_data/'+fileName+'.geojsonl','w') as outFile:
        for t in tweets:
            geojson = {
                'type':'Feature',
                'geometry':t['geo'],
                'properties':{
                    'user':t['actor']['preferredUsername'],
                    'time':t['postedTime'],
                    'text':t['body']
                }
            }
            outFile.write(json.dumps(geojson)+"\n")